## CNN 모델
### CNN : 챗봇 엔진에 문장 의도 분류를 위해 사용한다. 합성곱 신경망이라 불리며 이미지 분류에 좋은 성능을 가지고 있어 최근에 자율주행 자동차, 얼굴인식 등 이미지 판별 분야에서 많이 사용한다.
#### 합성곱 : 합성곱 필터라고 불리는 특정 크기의 행렬을 이미지 데이터(or 문장 데이터) 행렬에 슬라이딩 하면서 곱하고 더하는 연산을 말한다. 합성곱 연산은 필터가 더 이상 슬라이딩 할 수 없을 때까지 반복한다. 합성곱 필터는 다른 말로 마스크, 윈도우, 커널 등으로 불린다.
#### 특징맵 : 합성곱 연산을 통해 최종적으로 나온 결과를 말한다.
#### 패딩 : 출력의 크기를 조정하며 패딩된 곳은 0으로 채워진다.
#### 풀링 : 합성곱 연산 결과로 나온 특징맵의 크기를 줄이거나 주요한 특징을 추출하기 위해 사용하는 연산이다. 풀링 연산에는 최대 풀링과 평균 풀링 연산이 있으며 주로 최대 풀링 연산을 사용한다.

## 챗봇 문답 데이터 감정 분류 모델 by CNN

In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate

In [2]:
# 데이터 읽어오기
train_file = "./ChatbotData .csv"
data = pd.read_csv(train_file, delimiter=',')
features = data['Q'].tolist()
labels = data['label'].tolist()

In [7]:
# 단어 인덱스 시퀀스 벡터
corpus = [preprocessing.text.text_to_word_sequence(text) for text in features]
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(corpus)
sequences = tokenizer.texts_to_sequences(corpus)
word_index = tokenizer.word_index

MAX_SEQ_LEN = 15 # 단어 시퀀스 벡터 크기
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

In [9]:
# 훈련셋, 테스트셋, 검증셋 생성
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, labels))
ds = ds.shuffle(len(features))

train_size = int(len(padded_seqs)*0.7)
val_size = int(len(padded_seqs)*0.2)
test_size = int(len(padded_seqs)*0.1)

train_ds = ds.take(train_size).batch(20)
val_ds = ds.skip(train_size).take(val_size).batch(20)
test_ds = ds.skip(train_size+val_size).take(test_size).batch(20)

In [10]:
# 하이퍼파라미터 값 설정
dropout_prob = 0.5
EMB_SIZE = 128
EPOCH = 5
VOCAB_SIZE = len(word_index)+1 # 전체 단어 수

In [15]:
# CNN 모델 정의
input_layer = Input(shape=(MAX_SEQ_LEN,))
embedding_layer = Embedding(VOCAB_SIZE, EMB_SIZE, input_length=MAX_SEQ_LEN)(input_layer)
dropout_emb = Dropout(rate=dropout_prob)(embedding_layer)

conv1 = Conv1D(
    filters=128, 
    kernel_size=3, 
    padding='valid', 
    activation=tf.nn.relu)(dropout_emb)
pool1 = GlobalMaxPool1D()(conv1)

conv2 = Conv1D(
    filters=128, 
    kernel_size=4, 
    padding='valid', 
    activation=tf.nn.relu)(dropout_emb)
pool2 = GlobalMaxPool1D()(conv2)

conv3 = Conv1D(
    filters=128, 
    kernel_size=5, 
    padding='valid', 
    activation=tf.nn.relu)(dropout_emb)
pool3 = GlobalMaxPool1D()(conv3)

In [17]:
# 3, 4, 5-gram 이후 합치기
concat = concatenate([pool1, pool2, pool3])

hidden = Dense(128, activation=tf.nn.relu)(concat)
dropout_hidden = Dropout(rate=dropout_prob)(hidden)
logits = Dense(3, name='logits')(dropout_hidden)
predictions = Dense(3, activation=tf.nn.softmax)(logits)

In [20]:
# 모델 생성
model = Model(inputs=input_layer, outputs=predictions)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [21]:
# 모델 학습
model.fit(train_ds, validation_data=val_ds, epochs=EPOCH, verbose=1)

Epoch 1/5
414/414 [==============================] - 4s 10ms/step - loss: 0.8948 - accuracy: 0.5615 - val_loss: 0.5413 - val_accuracy: 0.8139
Epoch 2/5
414/414 [==============================] - 4s 9ms/step - loss: 0.5249 - accuracy: 0.8034 - val_loss: 0.2739 - val_accuracy: 0.9150
Epoch 3/5
414/414 [==============================] - 4s 9ms/step - loss: 0.2996 - accuracy: 0.8996 - val_loss: 0.1242 - val_accuracy: 0.9602
Epoch 4/5
414/414 [==============================] - 4s 9ms/step - loss: 0.1884 - accuracy: 0.9415 - val_loss: 0.1019 - val_accuracy: 0.9674
Epoch 5/5
414/414 [==============================] - 4s 9ms/step - loss: 0.1361 - accuracy: 0.9584 - val_loss: 0.0583 - val_accuracy: 0.9814


In [22]:
# 모델 평가
loss, accuracy = model.evaluate(test_ds, verbose=1)
print('Accuracy: %f'%(accuracy*100))
print('loss: %f'%(loss))

60/60 [==============================] - 0s 1ms/step - loss: 0.0590 - accuracy: 0.9856
Accuracy: 98.561758
loss: 0.058960


In [23]:
# 모델 저장
model.save('cnn_model.h5')

## 챗봇 무답 데이터 감정 분류 모델 사용

In [24]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model, load_model

In [25]:
# 데이터 읽어오기
train_file = "./ChatbotData .csv"
data = pd.read_csv(train_file, delimiter=',')
features = data['Q'].tolist()
labels = data['label'].tolist()

In [26]:
# 단어 인덱스 시퀀스 벡터
corpus = [preprocessing.text.text_to_word_sequence(text) for text in features]
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(corpus)
sequences = tokenizer.texts_to_sequences(corpus)

MAX_SEQ_LEN = 15 # 단어 시퀀스 벡터 크기
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

In [27]:
# 테스트셋 생성
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, labels))
ds = ds.shuffle(len(features))
test_ds = ds.take(2000).batch(20)

In [29]:
# 감정 분류 CNN 모델 불러오기
model = load_model('cnn_model.h5')
model.summary()
model.evaluate(test_ds, verbose=2)

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 15)]         0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 15, 128)      1715072     input_5[0][0]                    
__________________________________________________________________________________________________
dropout_4 (Dropout)             (None, 15, 128)      0           embedding_4[0][0]                
__________________________________________________________________________________________________
conv1d_4 (Conv1D)               (None, 13, 128)      49280       dropout_4[0][0]                  
_______________________________________________________________________________________

[0.05648089945316315, 0.4404999911785126]

In [30]:
# 테스트셋 10212번째 데이터 출력
print('단어 시퀀스 : ', corpus[10212])
print('단어 인덱스 시퀀스 : ', padded_seqs[10212])
print('문장 분류(정답) : ', labels[10212])

단어 시퀀스 :  ['썸', '타는', '여자가', '남사친', '만나러', '간다는데', '뭐라', '해']
단어 인덱스 시퀀스 :  [   13    61   127  4320  1333 12162   856    31     0     0     0     0
     0     0     0]
문장 분류(정답) :  2


In [31]:
# 테스트셋의 10212번째 데이터 감정 예측
picks = [10212]
predict = model.predict(padded_seqs[picks])
predict_class = tf.math.argmax(predict, axis=1)
print('감정 예측 점수 : ', predict)
print('감정 예측 클라스 : ', predict_class.numpy())

감정 예측 점수 :  [[1.5855716e-06 9.8004894e-06 9.9998856e-01]]
감정 예측 클라스 :  [2]
